In [13]:
import tensorflow as tf
from tensorflow import keras
from PIL import Image
import numpy as np
import time 
from datetime import datetime
from packaging import version
%load_ext tensorboard
import tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


## Load Model and Training Data 

In [6]:
local_data_dir = 'D:/card_data/cv_num_cards_deck/52kards'
img_height = 224
img_width = 224
img_size = (224, 224)
batch_size = 32
# Create the training dataset
train_ds = tf.keras.utils.image_dataset_from_directory(
    local_data_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(img_height, img_width), # specified the resizing 
    batch_size=batch_size) 

Found 12304 files belonging to 52 classes.
Using 9844 files for training.


In [7]:
# Create the validation dataset
val_ds = tf.keras.utils.image_dataset_from_directory(
    local_data_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)

Found 12304 files belonging to 52 classes.
Using 2460 files for validation.


In [15]:
normalization_layer = tf.keras.layers.Rescaling(1./255) # 1./127.5, offset=-1
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))

In [8]:
class_names = train_ds.class_names

In [3]:
MODEL_PATH = 'models/model_to_use'
model = tf.keras.models.load_model(MODEL_PATH)

## Graph Layers 

In [27]:
# Define the Keras TensorBoard callback.
logdir="logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

# Train the model.
model.fit(
    train_ds,
    batch_size=batch_size,
    epochs=2,
    validation_data=val_ds,
    callbacks=[tensorboard_callback])

Epoch 1/2
308/308 [==============================] - 160s 518ms/step - loss: 0.0205 - accuracy: 0.9939 - val_loss: 0.0746 - val_accuracy: 0.9805
Epoch 2/2
308/308 [==============================] - 155s 501ms/step - loss: 0.0306 - accuracy: 0.9911 - val_loss: 0.0932 - val_accuracy: 0.9724


In [18]:
%tensorboard --logdir logs

In [19]:
def holdout_set(model):
    """
    Test a model against the holdout set 
    """
    total = 0 
    correct = 0 
    test_dir = "D:/card_data/cv_num_cards_deck/test_images/"
    predics = []
    for card in os.listdir(test_dir):
        card_path = os.path.join(test_dir, card)
        images = os.listdir(card_path)
        for image in images:
            total += 1
            image_path = os.path.join(card_path, image)
            predict = image_prediction(image_path, model, class_names)
            if card == predict:
                correct += 1

            
            predics.append((card, predict))
    print(predics)
    print(f"overall accuracy is {round((correct/total)*100, 2)}%. So {correct} correct out of {total}")

In [23]:
def image_prediction(image_path, model, class_names, img_width=224, img_height=224):
    image = PIL.Image.open(image_path).resize((img_width, img_height)).convert('RGB')
    image_array = np.array(image) / 255  # Scale pixel values to -1 to 1
    image_array = np.expand_dims(image_array, axis=0)  # Add a batch dimension

    predictions = model.predict(image_array)
    
    # Since the model's final layer uses a softmax activation, predictions are already in probability form
    predicted_class = np.argmax(predictions[0])  # Get the index of the highest probability class
    probability = predictions[0][predicted_class]  # Get the probability of the predicted class
    
    print(f"Predicted class: {predicted_class}, Probability: {probability}, Class: {class_names[predicted_class]}")
    return class_names[predicted_class]

In [25]:
import os
import PIL

In [28]:
holdout_set(model)

1/1 [==============================] - 0s 25ms/step
Predicted class: 0, Probability: 1.0, Class: 10c
1/1 [==============================] - 0s 25ms/step
Predicted class: 1, Probability: 0.9999966621398926, Class: 10d
1/1 [==============================] - 0s 24ms/step
Predicted class: 6, Probability: 0.9212932586669922, Class: 2h
1/1 [==============================] - 0s 24ms/step
Predicted class: 4, Probability: 0.9992203712463379, Class: 2c
1/1 [==============================] - 0s 24ms/step
Predicted class: 7, Probability: 1.0, Class: 2s
1/1 [==============================] - 0s 24ms/step
Predicted class: 10, Probability: 1.0, Class: 3h
1/1 [==============================] - 0s 24ms/step
Predicted class: 11, Probability: 0.9327682852745056, Class: 3s
1/1 [==============================] - 0s 23ms/step
Predicted class: 12, Probability: 1.0, Class: 4c
1/1 [==============================] - 0s 24ms/step
Predicted class: 15, Probability: 0.9985564351081848, Class: 4s
1/1 [==============